In [ ]:
# Prompt:
# You are an analyst for a premium travel agency. The company has tasked you to analyze the recent bookings they had with some of their clients. Each row represents the transaction of a person of interest (POI) in the company. Some details regarding the POI is also given, such as which luxury group they are from, as well as their individual net worth. Most importantly, you also know their route which displays the order of their travels from different points A to E.

# A sample route of EBDAC means that:

# From starting point E, they went to point B.
# From point B, they went to point D.
# From point D, they went to point A.
# From point A, they went to ending point C.
# You are tasked to come up with the following:
# # How many POIs are there in each group?
# # Using the route_matrix.json file, calculate each POI's travel expenditure.
# Due to company policy, add a column named budget to denote if a person is overbudget (if their travel expenses go over 5% of their net worth). If they are overbudget, place them as "Over Budget"; else, place them as "Within Budget".
# Which group has the highest average net worth?
# Which group has the highest % of people who are Over Budget?
# Which group had the most direct flights from point A to point B (or from B to A)?

In [79]:
import pandas as pd
import numpy as np

matrix_df = pd.read_json('route_matrix.json')
poi_df = pd.read_csv('poi_dataset.csv')


In [2]:
# 1. How many POIs are there in each group?
poi_df.groupby('group').count()['poi_number']

group
Group 1     6615
Group 10    6544
Group 11    6603
Group 12    6600
Group 13    6558
Group 14    6564
Group 15    6562
Group 16    6555
Group 17    6445
Group 18    6565
Group 19    6546
Group 2     6533
Group 20    6517
Group 3     6656
Group 4     6660
Group 5     6571
Group 6     6551
Group 7     6501
Group 8     6572
Group 9     6645
Name: poi_number, dtype: int64

In [37]:
# 2. Using the route_matrix.json file, calculate each POI's travel expenditure
def POI_expense(route):
    total = 0
    for i in range(4):
        total += matrix_df.loc[route[i],route[i+1]]
    return total

poi_df['travel expenditure'] = poi_df['route'].apply(POI_expense)
poi_df['fivepercent'] = poi_df['net_worth'].apply(lambda x: x*0.05)



In [51]:
# 3. Due to company policy, add a column named budget to denote if a person is overbudget (if their travel expenses go over 5% of their net worth). 
# If they are overbudget, place them as "Over Budget"; else, place them as "Within Budget"
import numpy as np
# add budget column
poi_df['budget'] = np.where((poi_df['travel expenditure'] < poi_df['fivepercent']),'Within Budget','Over Budget')
poi_df

,poi_number,group,route,net_worth,fivepercent,travel expenditure,AB,budget
0,0,Group 1,EBDAC,459,22.95,16,0,Within Budget
1,1,Group 3,ABCED,407,20.35,22,1,Over Budget
2,2,Group 9,BECAD,5,0.25,21,0,Over Budget
3,3,Group 1,BEACD,832,41.60,29,0,Within Budget
4,4,Group 1,CBDAE,928,46.40,8,0,Within Budget
...,...,...,...,...,...,...,...,...
131358,131358,Group 2,AECBD,755,37.75,9,0,Within Budget
131359,131359,Group 9,EADCB,143,7.15,11,0,Over Budget
131360,131360,Group 5,CBAED,202,10.10,18,1,Over Budget
131361,131361,Group 3,ADCEB,277,13.85,17,0,Over Budget


In [21]:
# 4. Which group has the highest average net worth?
poi_df.groupby('group').mean(numeric_only=True)['net_worth'].sort_values(ascending=False)

group
Group 10    510.857427
Group 13    506.817627
Group 9     506.086832
Group 4     506.020871
Group 18    505.240061
Group 19    504.931714
Group 20    504.469695
Group 6     503.275836
Group 3     503.217849
Group 17    502.266253
Group 7     501.841870
Group 2     500.622991
Group 5     500.229037
Group 15    499.502133
Group 1     498.621920
Group 16    497.430206
Group 14    496.734918
Group 12    495.620152
Group 8     494.873098
Group 11    493.068605
Name: net_worth, dtype: float64

In [77]:
# 5. Which group has the highest % of people who are Over Budget?
edited = (poi_df.groupby(['group','budget']).count()['poi_number'])/poi_df.groupby(['group']).count()['poi_number']
new_df = pd.DataFrame(edited)
new_df.reset_index(inplace=True)
new_df[(new_df['budget']=='Over Budget')].sort_values(by=['poi_number'],ascending=False)

group     budget       
Group 1   Over Budget      0.367952
          Within Budget    0.632048
Group 10  Over Budget      0.349939
          Within Budget    0.650061
Group 11  Over Budget      0.365137
          Within Budget    0.634863
Group 12  Over Budget      0.367879
          Within Budget    0.632121
Group 13  Over Budget      0.357121
          Within Budget    0.642879
Group 14  Over Budget      0.359385
          Within Budget    0.640615
Group 15  Over Budget      0.365285
          Within Budget    0.634715
Group 16  Over Budget      0.359115
          Within Budget    0.640885
Group 17  Over Budget      0.352521
          Within Budget    0.647479
Group 18  Over Budget      0.352932
          Within Budget    0.647068
Group 19  Over Budget      0.351665
          Within Budget    0.648335
Group 2   Over Budget      0.359100
          Within Budget    0.640900
Group 20  Over Budget      0.350928
          Within Budget    0.649072
Group 3   Over Budget      0.353215
    

In [53]:
# 6. Which group had the most direct flights from A to B (or from B to A)?
poi_df['AB']=poi_df['route'].apply(lambda x: 1 if "AB" in x or "BA" in x else(0))
poi_df.groupby('group').sum(numeric_only=True)['AB'].sort_values(ascending=False)


group
Group 3     2742
Group 13    2717
Group 18    2665
Group 12    2659
Group 6     2653
Group 9     2649
Group 19    2640
Group 5     2637
Group 15    2636
Group 2     2629
Group 8     2617
Group 7     2614
Group 1     2611
Group 16    2608
Group 20    2604
Group 10    2602
Group 11    2597
Group 4     2589
Group 14    2574
Group 17    2508
Name: AB, dtype: int64

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=83c9bdbf-5eca-41c1-993f-c33735cc8b28' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>